In [1]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [2]:
from IPython.display import Markdown
import json

from helpers.functions import *
from helpers.chat import *

In [3]:
profile_summary = """
Name: Mumtaz
Occupation: Junior Software Engineer

As a Junior Software Engineer, I am passionate about leveraging technology to solve complex problems and improve user experiences. I have a strong foundation in software development principles and a keen interest in expanding my knowledge and skills in various domains.

Learning Interests:
I am particularly interested in the following areas:

Statistics: Understanding data analysis techniques to derive meaningful insights.
Machine Learning (ML): Exploring algorithms and models that enable systems to learn from data.
Artificial Intelligence (AI): Delving into intelligent systems that mimic human behavior and decision-making.
Large Language Models (LLMs): Investigating advanced language processing technologies to enhance natural language understanding and generation.
I am eager to further develop my expertise in these fields and apply them to real-world applications, contributing to innovative projects and technologies."""

In [4]:
stage_name = "discuss"

In [5]:
stage_config = get_yaml_data(stage_name)

In [6]:
extract_system_message = create_system_message(stage_name, stage_config, mode='extract')
inital_user_message    = {"role":"user","content": f"Hey sharing my summary here. {profile_summary}"}

In [7]:
required_fields = [f['title'] for f in stage_config['fields']]
required_fields

['interest_areas', 'learning_style', 'modules', 'timeline', 'goals_alignment']

In [16]:
messages = [extract_system_message,inital_user_message]
messages

[{'role': 'system',
  'content': 'You are an highly attentive assistant to Zavmo - a conversational AI assistant.\n\nZavmo will probe a learner for their profile details through a series of engaging and interactive questions. The learner will answer these questions to the best of their ability. \n\nYour job is to extract the information from the learner\'s answers, and build a profile for them, and present it to Zavmo. You will also guide Zavmo about what questions to ask next, or what actions to take.\n\nAt each interaction, you will be provided with the information we currently have about the learner, and the information that\'s still missing.\n\nYour task is to extract the information from the learner\'s answers, and build a profile for them, and present it to Zavmo. You will also guide Zavmo about what actions to take next.\n\n### Conversation Flow\n\nLet\'s approach this step by step:\n\n- First, review the information we currently have about the learner, and the information that\

In [8]:
def get_modified_json(curriculum):
    markdown_output = ""
    
    for subject in curriculum:
        # Add subject name and duration
        markdown_output += f"- **{subject['subject_name']}** - Duration: {subject['duration']}\n"
        
        # Add modules under each subject
        markdown_output += "  - Modules:\n"
        for module in subject['modules']:
            markdown_output += f"    - {module}\n"
    
    return markdown_output

In [21]:
from helpers.tools.discuss import *


class DiscussCollector(BaseModel):
    interest_areas: GetInterestAreas
    learning_style: GetLearningStyle
    curriculum: GetCurriculum
    timeline: GetTimeline
    goals_alignment: GetGoalsAlignment
    response: str = Field(description="Generate a response to whatever the user responds. Ask the questions step by step and keep the interaction based on instructions given")


In [26]:
def get_response_and_fields(response_dict):
    # if response_dict['curriculum']['curriculum']:
    #     return (f"Please confirm for the below shared curriculum plan\n{get_modified_json(response_dict['curriculum']['curriculum'])}",response_dict) 
    return (response_dict['response'],response_dict)

In [31]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    response_format=DiscussCollector,
)

completion = completion.choices[0].message.parsed 
response_dict = completion.dict()
response,fields = get_response_and_fields(response_dict)

In [32]:
Markdown(response)

Great! Your curriculum plan has been confirmed. Let's move to the next stage.

In [30]:
messages.extend([{"role":"assistant","content":response},
                {"role":"user","content":"Yeah works"}])   ## enter your input here

In [12]:
messages

[{'role': 'system',
  'content': 'You are an highly attentive assistant to Zavmo - a conversational AI assistant.\n\nZavmo will probe a learner for their profile details through a series of engaging and interactive questions. The learner will answer these questions to the best of their ability. \n\nYour job is to extract the information from the learner\'s answers, and build a profile for them, and present it to Zavmo. You will also guide Zavmo about what questions to ask next, or what actions to take.\n\nAt each interaction, you will be provided with the information we currently have about the learner, and the information that\'s still missing.\n\nYour task is to extract the information from the learner\'s answers, and build a profile for them, and present it to Zavmo. You will also guide Zavmo about what actions to take next.\n\n### Conversation Flow\n\nLet\'s approach this step by step:\n\n- First, review the information we currently have about the learner, and the information that\

# In the begining

In [20]:
response_dict

{'interest_areas': {'interest_areas': 'Statistics, Machine Learning, Artificial Intelligence, Large Language Models'},
 'learning_style': {'learning_style': 'none'},
 'curriculum': {'curriculum': []},
 'timeline': {'timeline': 'none'},
 'goals_alignment': {'goals_alignment': 'none'},
 'response': "Thank you for sharing your areas of interest, Mumtaz! It's great to see your enthusiasm for these fields. Now, let’s talk about your preferred learning style. Do you find that you learn best through visual aids, listening, or hands-on projects?"}

# After finishing the stage

In [33]:
response_dict

{'interest_areas': {'interest_areas': 'Statistics, Machine Learning, Artificial Intelligence, Large Language Models'},
 'learning_style': {'learning_style': 'Kinesthetic'},
 'curriculum': {'curriculum': [{'subject_name': 'Data Science',
    'modules': ['Statistics', 'Machine Learning'],
    'duration': '2 weeks'},
   {'subject_name': 'Artificial Intelligence',
    'modules': ['AI Fundamentals',
     'Large Language Models',
     'Advanced LLM Techniques'],
    'duration': '2 weeks'}]},
 'timeline': {'timeline': '4 weeks'},
 'goals_alignment': {'goals_alignment': 'The proposed plan aligns with my goals.'},
 'curriculum_confirmaion': <GetCurriculumConfirmation.yes: 'Yes'>,
 'response': "Great! Your curriculum plan has been confirmed. Let's move to the next stage."}